In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from random import randint
import warnings
import cv2
import time
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

### Porbability tables and arrays class

In [3]:
class PorbabilityClass():
    
   

    
    def __init__(self, aisle, weekday):
        self.weekday = weekday
        self.aisle = aisle
    
    
    def read_data(self):
        df = pd.read_csv('friday.csv', delimiter=";")
        for csv in os.listdir():
            if csv[-3:] == "csv":                
                df_day = pd.read_csv(f'{csv}', delimiter=";")
                df = pd.concat([df, df_day], sort=True)
        return df        
           
    
    def add_datetime_columns(self):
        df =  self.read_data()
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df['hour'] = df['timestamp'].dt.hour
        df['weekday'] = df['timestamp'].dt.weekday
        df['weekday'].replace({0:'Mon',1:'Tues', 2:'Wednes',3:'Thurs',4:'Fri'}, inplace=True)
        
        return df
    
        
        
    def porb_matrix_by_day(self):
        df = self.add_datetime_columns()
        if self.weekday == 'all_days':       
            df_1 = pd.DataFrame(df.sort_values(['customer_no','timestamp']).groupby(['weekday','customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1 = pd.DataFrame(df_1.index.get_level_values(2),df_1['timestamp']).reset_index()
            
        else:
            df_1 = df[df['weekday']== self.weekday]
            df_1 = pd.DataFrame(df_1.sort_values(['customer_no','timestamp']).groupby(['customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1 = pd.DataFrame(df_1.index.get_level_values(1),df_1['timestamp']).reset_index()   


        df_1['next_location'] = df_1['location'].shift(-1)
        df_1['next_location'][df_1['location']=='checkout']= 'checkout'
        df_1['initial'] = df_1['location'].shift(-1)

        first_loc = df_1['initial'][df_1['location']=='checkout']
        first_loc = first_loc.value_counts()/first_loc.value_counts().sum()

        prob_mat = pd.crosstab(df_1['location'], df_1['next_location'], normalize=0)

        return first_loc, prob_mat 
    
        
        
        
    def sector_time_prob(self):
        df = self.add_datetime_columns()
        if self.weekday == 'all_days': 
            df_1 = pd.DataFrame(df.sort_values(['customer_no','timestamp']).groupby(['weekday','customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1= pd.DataFrame(df_1.index.get_level_values(2), df_1['timestamp']).reset_index()
        else:
            df_1 = df[df['weekday'] == self.weekday]
            df_1 = pd.DataFrame(df_1.sort_values(['customer_no','timestamp']).groupby(['customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1= pd.DataFrame(df_1.index.get_level_values(1), df_1['timestamp']).reset_index()
       
        df_1['timestamp-2'] = df_1['timestamp'].shift(-1)        
        df_1= df_1[df_1['location'] == self.aisle]
        df_1['timespent'] = df_1['timestamp-2'] - df_1['timestamp']
        df_1 = df_1[(df_1['timespent']< pd.to_timedelta('40m')) & (df_1['timespent'] >= pd.to_timedelta('1m'))]

        time_prob = df_1['timespent'].value_counts()/df_1['timespent'].value_counts().sum()
        time_prob.index = time_prob.index/pd.Timedelta('1 min')

        return time_prob
 

### Customer Class

In [242]:
class Customer(PorbabilityClass):
    """ This is a customer class """
    
    def __init__(self, aisle, weekday, image):
        PorbabilityClass.__init__(self, aisle, weekday)
        self.image = image
        self.location = np.array((randint(580, 620), randint(700, 780)))
        self.h = self.image.shape[0]
        self.w = self.image.shape[1]
        self.payment = 0
        self.shopping_hist = []
          
    def aisle_pattern(self):
        #first time your customer arrives, place her in the 'first_aisle - every aisle except checkout'
        initial_state, prob_mat = self.porb_matrix_by_day()
        aisle = random.choices(initial_state.index, initial_state)[0]
        yield aisle
        #for every other time until she checkouts, place her wherever the aisle_activity prob matrix places her
        while True:
            aisle = random.choices(prob_mat.index, prob_mat.loc[aisle])[0]
            yield aisle
            
    def move(self):
        '''
        The method move lets the customers move in the supermarket
        '''
        self.aisles_locs= {'fruit':(180,750), 'spices':(150,570), 'drinks':(280,140), 'dairy':(130,335), 'checkout':(600,220)}
        
        
        
        
#         for i in range(len(self.shopping_hist)):
        y, x = self.location
   
        self.target_position = np.array(self.aisles_locs[self.shopping_hist[0]])
        ty, tx = self.target_position          
        # go up
        if x != tx and y != 60:
            trajectory = 60 - y
            if trajectory > 0:
                self.location[0] += 1
            elif trajectory < 0:
                self.location[0] -= 1

        # go right or left
        if x != tx and y == 60:
            trajectory = tx - x
            if trajectory > 0:
                self.location[1] += 1
            if trajectory < 0:
                self.location[1] -= 1

        # go down
        if x == tx:
            trajectory = ty - y
            if trajectory > 0:
                self.location[0] += 1
            if trajectory < 0:
                self.location[0] -= 1
                
        if self.location[0] == self.target_position[0]  and self.location[1] == self.target_position[1]:
            self.target_position = np.array(self.aisles_locs[self.shopping_hist[1]])
            
            if x != tx and y != 60:
                trajectory = 60 - y
                if trajectory > 0:
                    self.location[0] += 1
                elif trajectory < 0:
                    self.location[0] -= 1

            # go right or left
            if x != tx and y == 60:
                trajectory = tx - x
                if trajectory > 0:
                    self.location[1] += 1
                if trajectory < 0:
                    self.location[1] -= 1

            # go down
            if x == tx:
                trajectory = ty - y
                if trajectory > 0:
                    self.location[0] += 1
                if trajectory < 0:
                    self.location[0] -= 1
 

    def money_spent(self):
        for section in self.shopping_hist:
            if section == 'dairy':
                self.aisle = 'dairy'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 5 * time_spent
                self.payment  = self.payment + pay
            if section == 'fruit':
                self.aisle = 'fruit'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 4 * time_spent
                self.payment  = self.payment + pay
            if section == 'spices':
                self.aisle = 'spices'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 3 * time_spent
                self.payment  = self.payment + pay
            if section == 'drinks':
                self.aisle = 'drinks'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 6 * time_spent
                self.payment  = self.payment + pay     
            else:
                self.payment  = self.payment

    def pattern(self):        
        pattern_func = self.aisle_pattern()
        while True:
            next_step = next(pattern_func)
            self.shopping_hist.append(next_step)
            if next_step =='checkout':
                break


In [243]:
class SupermarketSim(Customer):
    
    def __init__(self, background, customer):
        self.background = background
        self.customers = customers
        self.frame = background
        
    def draw(self):
        self.frame = self.background.copy()
        for customer in customers:
            y, x = customer.location
            self.frame[y:y+customer.h, x:x+customer.w] = customer.image

    def run_one_iteration(self):
        for customer in customers:
            customer.move()
            
            


In [250]:
customers = []
for _ in range(2):
    c = Customer(weekday='all_days', aisle='fruit', image= cv2.imread('customer_2.png'))
    c.pattern()
    c.money_spent()
    customers.append(c)


C:\Users\Khamirz\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [251]:
customers[1].shopping_hist

['drinks', 'spices', 'checkout']

In [252]:
customers[1].move()

[614 779]


In [253]:
sim = SupermarketSim(cv2.imread('market.png'), customers )

In [254]:
img = cv2.imread('market.png')

In [255]:
while True:
    frame = img.copy()

    sim.draw()
    sim.run_one_iteration()


    cv2.imshow('frame', sim.frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()


[584 749]
[613 779]
[583 749]
[612 779]
[582 749]
[611 779]
[581 749]
[610 779]
[580 749]
[609 779]
[579 749]
[608 779]
[578 749]
[607 779]
[577 749]
[606 779]
[576 749]
[605 779]
[575 749]
[604 779]
[574 749]
[603 779]
[573 749]
[602 779]
[572 749]
[601 779]
[571 749]
[600 779]
[570 749]
[599 779]
[569 749]
[598 779]
[568 749]
[597 779]
[567 749]
[596 779]
[566 749]
[595 779]
[565 749]
[594 779]
[564 749]
[593 779]
[563 749]
[592 779]
[562 749]
[591 779]
[561 749]
[590 779]
[560 749]
[589 779]
[559 749]
[588 779]
[558 749]
[587 779]
[557 749]
[586 779]
[556 749]
[585 779]
[555 749]
[584 779]
[554 749]
[583 779]
[553 749]
[582 779]
[552 749]
[581 779]
[551 749]
[580 779]
[550 749]
[579 779]
[549 749]
[578 779]
[548 749]
[577 779]
[547 749]
[576 779]
[546 749]
[575 779]
[545 749]
[574 779]
[544 749]
[573 779]
[543 749]
[572 779]
[542 749]
[571 779]
[541 749]
[570 779]
[540 749]
[569 779]
[539 749]
[568 779]
[538 749]
[567 779]
[537 749]
[566 779]
[536 749]
[565 779]
[535 749]
[564 779]


[146 749]
[175 779]
[145 749]
[174 779]
[144 749]
[173 779]
[143 749]
[172 779]
[142 749]
[171 779]
[141 749]
[170 779]
[140 749]
[169 779]
[139 749]
[168 779]
[138 749]
[167 779]
[137 749]
[166 779]
[136 749]
[165 779]
[135 749]
[164 779]
[134 749]
[163 779]
[133 749]
[162 779]
[132 749]
[161 779]
[131 749]
[160 779]
[130 749]
[159 779]
[129 749]
[158 779]
[128 749]
[157 779]
[127 749]
[156 779]
[126 749]
[155 779]
[125 749]
[154 779]
[124 749]
[153 779]
[123 749]
[152 779]
[122 749]
[151 779]
[121 749]
[150 779]
[120 749]
[149 779]
[119 749]
[148 779]
[118 749]
[147 779]
[117 749]
[146 779]
[116 749]
[145 779]
[115 749]
[144 779]
[114 749]
[143 779]
[113 749]
[142 779]
[112 749]
[141 779]
[111 749]
[140 779]
[110 749]
[139 779]
[109 749]
[138 779]
[108 749]
[137 779]
[107 749]
[136 779]
[106 749]
[135 779]
[105 749]
[134 779]
[104 749]
[133 779]
[103 749]
[132 779]
[102 749]
[131 779]
[101 749]
[130 779]
[100 749]
[129 779]
[ 99 749]
[128 779]
[ 98 749]
[127 779]
[ 97 749]
[126 779]


[149 570]
[150 570]
[600 220]
[ 60 484]
[151 570]
[150 570]
[600 220]
[ 60 483]
[149 570]
[150 570]
[600 220]
[ 60 482]
[151 570]
[150 570]
[600 220]
[ 60 481]
[149 570]
[150 570]
[600 220]
[ 60 480]
[151 570]
[150 570]
[600 220]
[ 60 479]
[149 570]
[150 570]
[600 220]
[ 60 478]
[151 570]
[150 570]
[600 220]
[ 60 477]
[149 570]
[150 570]
[600 220]
[ 60 476]
[151 570]
[150 570]
[600 220]
[ 60 475]
[149 570]
[150 570]
[600 220]
[ 60 474]
[151 570]
[150 570]
[600 220]
[ 60 473]
[149 570]
[150 570]
[600 220]
[ 60 472]
[151 570]
[150 570]
[600 220]
[ 60 471]
[149 570]
[150 570]
[600 220]
[ 60 470]
[151 570]
[150 570]
[600 220]
[ 60 469]
[149 570]
[150 570]
[600 220]
[ 60 468]
[151 570]
[150 570]
[600 220]
[ 60 467]
[149 570]
[150 570]
[600 220]
[ 60 466]
[151 570]
[150 570]
[600 220]
[ 60 465]
[149 570]
[150 570]
[600 220]
[ 60 464]
[151 570]
[150 570]
[600 220]
[ 60 463]
[149 570]
[150 570]
[600 220]
[ 60 462]
[151 570]
[150 570]
[600 220]
[ 60 461]
[149 570]
[150 570]
[600 220]
[ 60 460]


[151 570]
[150 570]
[600 220]
[ 60 243]
[149 570]
[150 570]
[600 220]
[ 60 242]
[151 570]
[150 570]
[600 220]
[ 60 241]
[149 570]
[150 570]
[600 220]
[ 60 240]
[151 570]
[150 570]
[600 220]
[ 60 239]
[149 570]
[150 570]
[600 220]
[ 60 238]
[151 570]
[150 570]
[600 220]
[ 60 237]
[149 570]
[150 570]
[600 220]
[ 60 236]
[151 570]
[150 570]
[600 220]
[ 60 235]
[149 570]
[150 570]
[600 220]
[ 60 234]
[151 570]
[150 570]
[600 220]
[ 60 233]
[149 570]
[150 570]
[600 220]
[ 60 232]
[151 570]
[150 570]
[600 220]
[ 60 231]
[149 570]
[150 570]
[600 220]
[ 60 230]
[151 570]
[150 570]
[600 220]
[ 60 229]
[149 570]
[150 570]
[600 220]
[ 60 228]
[151 570]
[150 570]
[600 220]
[ 60 227]
[149 570]
[150 570]
[600 220]
[ 60 226]
[151 570]
[150 570]
[600 220]
[ 60 225]
[149 570]
[150 570]
[600 220]
[ 60 224]
[151 570]
[150 570]
[600 220]
[ 60 223]
[149 570]
[150 570]
[600 220]
[ 60 222]
[151 570]
[150 570]
[600 220]
[ 60 221]
[149 570]
[150 570]
[600 220]
[ 60 220]
[151 570]
[150 570]
[600 220]
[ 60 219]


[149 570]
[150 570]
[600 220]
[198 140]
[151 570]
[150 570]
[600 220]
[199 140]
[149 570]
[150 570]
[600 220]
[200 140]
[151 570]
[150 570]
[600 220]
[201 140]
[149 570]
[150 570]
[600 220]
[202 140]
[151 570]
[150 570]
[600 220]
[203 140]
[149 570]
[150 570]
[600 220]
[204 140]
[151 570]
[150 570]
[600 220]
[205 140]
[149 570]
[150 570]
[600 220]
[206 140]
[151 570]
[150 570]
[600 220]
[207 140]
[149 570]
[150 570]
[600 220]
[208 140]
[151 570]
[150 570]
[600 220]
[209 140]
[149 570]
[150 570]
[600 220]
[210 140]
[151 570]
[150 570]
[600 220]
[211 140]
[149 570]
[150 570]
[600 220]
[212 140]
[151 570]
[150 570]
[600 220]
[213 140]
[149 570]
[150 570]
[600 220]
[214 140]
[151 570]
[150 570]
[600 220]
[215 140]
[149 570]
[150 570]
[600 220]
[216 140]
[151 570]
[150 570]
[600 220]
[217 140]
[149 570]
[150 570]
[600 220]
[218 140]
[151 570]
[150 570]
[600 220]
[219 140]
[149 570]
[150 570]
[600 220]
[220 140]
[151 570]
[150 570]
[600 220]
[221 140]
[149 570]
[150 570]
[600 220]
[222 140]


[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]


[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]


[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
[280 140]
[150 570]
[151 570]
[150 570]
[600 220]
[279 140]
[280 140]
[150 570]
[149 570]
[150 570]
[600 220]
[281 140]
